# Qualtrics Survey Translator with OpenAI GPT-4

This project provides a Python script to translate Qualtrics survey content into multiple target languages using OpenAI's GPT-4 API. It automates the translation workflow by combining survey questions and answer options for better context, translating via GPT-4, and generating files ready for proofreading and import back into Qualtrics.


In [2]:
# Import libraries and set display options

import pandas as pd
from datetime import datetime
today=f"{datetime.now():%Y-%m-%d}"

%load_ext autoreload
%autoreload 2
def d(l):
    pd.set_option('display.max_colwidth', l)

In [3]:
# Load the original Qualtrics survey export CSV file
# Display a random sample of 3 rows to verify data structure

df=pd.read_csv("translation/Narratives of the European Energy Crisis-EN.csv")
df.sample(3)

,PhraseID,EN,NL
143,QID18_Choice2,Keeping energy costs manageable for consumers ...,Het beheersbaar houden van energiekosten voor ...
90,QID13_Choice2,"No, it is mostly over.","Nee, deze is voor het grootste deel voorbij."
158,QID42_Choice3,While there were conflicts and misunderstandin...,NaN


In [4]:
# Setup OpenAI GPT-4 API client with credentials stored in separate file
# Define prompt template for GPT-4 to translate survey questions professionally
# Define a function to send text to GPT-4 and receive the translated output

from openai import OpenAI

import openAI_key
client = OpenAI(
  api_key=openAI_key.key,
  organization=openAI_key.organization[0],
  project=openAI_key.project[0]
)

def ChatBot_intro(target_lang, survey_topic="European Energy Crisis"):
    return f"""You are expert in the formulating questions of European surveys. 
    You help me translate a survey about the {survey_topic} from English to {target_lang}.
    Use a professional tone and language that typical for {target_lang} survey questions, and accessible to citizens.
    Only return the translation and not other comments but keep the HTML code.
    Ensure the language is correct and professional. Now translate the following text: """


def OpenAI_translation(question_text, target_lang="German"):
    if isinstance(question_text,str):
        if len(question_text)>5:
            print(question_text[:10], end=" // ")
            completion = client.chat.completions.create(
            model="gpt-4.1",
            messages=[{"role": "system", "content": ChatBot_intro(target_lang)},
                {"role": "user","content": question_text}],
            temperature=0)
    
            result=completion.choices[0].message.content
            return result
        else: 
            return ""
    else:
        return ""

In [5]:
# Define supported target languages and their corresponding Qualtrics language codes

Lang_dict = {
    "English": "EN",
    "French": "FR",
    "Czech": "CS",
    "Polish": "PL",
    "Romanian": "RO",
    "Spanish": "ES",
    "Dutch": "NL",
    "Italian": "IT",
    "German": "DE",
    "Portuguese":"PT"
}

In [6]:
# Functions to translate and save survey text, then rebuild translations into Qualtrics format


shit=["""```""","&nbsp;", "html"] # strings to remove from translations

def translate_and_save(df, target_lang="German",sep="<SEP>"):

    # Split 'PhraseID' into two parts: first and second (if exists)
    split_qid = df['PhraseID'].str.split('_', n=1, expand=True)
    df['QID_first'] = split_qid[0]
    df['QID_second'] = split_qid[1].fillna('')  # fill missing second part with empty string
    
    # Remove unwanted HTML tags/spans and non-breaking spaces from English text
    import re
    df["EN"]=df["EN"].dropna().apply(lambda x: re.sub(r'</?span\b[^>]*>|<br\s*/?>|&nbsp;', '', x))
    
   # Group English texts by main QID and join with separator
    Source = df.groupby('QID_first', sort=False)['EN'].apply(lambda x: sep.join(x.astype(str)))

    print(target_lang,datetime.now())
    # Translate grouped text using GPT-4
    Target = Source.apply(OpenAI_translation, target_lang=target_lang)

    # Clean unwanted strings from translations
    for i in shit:
        Target=Target.str.replace(i,"").str.strip()

    # Combine source and target translations into a DataFrame
    df_trans=pd.DataFrame([Source,Target]).T
    lang_column_name=Lang_dict[target_lang]

    # Save translation draft for proofreading
    df_trans.columns=["EN",lang_column_name]
    translation_file_name=f"translation/{today} Survey translation {lang_column_name}.xlsx"
    df_trans.to_excel(translation_file_name)
    
    print("saved to:", translation_file_name)
    return Target
    
    
def rebuild_translation_file(df,Target=0,target_lang="German",sep="<SEP>"):
    """ Load the proofread Excel translation file (if Target=0), split translations back into individual rows, 
    map translations back to original PhraseIDs, and save a Qualtrics-compatible CSV file. """ 
    
    lang_column_name=Lang_dict[target_lang]

    if isinstance(Target,int):
        Target=pd.read_excel(f'translation/2025-07-21 Survey translation {lang_column_name}.xlsx')
        Target=Target[lang_column_name]
        # Split the translated text back into parts and explode to get one part per row
    translated_parts = Target.str.split(sep).explode()
    
    # Reconstruct QID for matching: combine first and second parts (if second exists)
    reconstructed_qid = df.apply(
        lambda row: f"{row['QID_first']}_{row['QID_second']}" if row['QID_second'] else row['QID_first'],
        axis=1
    )
    
    translated_parts.index = df.index
    
    df[lang_column_name] = translated_parts
    df.drop(columns=['QID_first', 'QID_second'], inplace=True)

    filename=f"translation/Narratives of the European Energy Crisis-EN-{lang_column_name}_filled.csv"
    df[["PhraseID","EN",lang_column_name]].to_csv(filename,index=False, encoding='utf8')

    print("saved to:", filename)



In [7]:
# Example usage: run translation and rebuild functions for a single language
# Uncomment and modify target_lang as needed

# Target=translate_and_save(df, target_lang="Portuguese",sep=" <hr /> ")
# rebuild_translation_file(df,Target=0,target_lang="Portuguese",sep=" <hr /> ")

In [11]:
# Batch translation loop over selected languages (uncomment to run)

for lang in list(Lang_dict):
    print(lang)
   # Target=translate_and_save(df, target_lang=lang,sep=" <hr /> ")
   # rebuild_translation_file(df,Target=0,target_lang=lang,sep=" <hr /> ")


English
French
Czech
Polish
Romanian
Spanish
Dutch
Italian
German
Portuguese


In [12]:
# Convert the Excel translation files into Word documents for easier proofreading
# Uses htmldocx to preserve HTML formatting during conversion
# Proofreaders should update the Excel files and re-run rebuild_translation_file to finalize

import os
import glob
import pandas as pd
from docx import Document
from htmldocx import HtmlToDocx
import re

#!pip install htmldocx

folder = "translation"
pattern = os.path.join(folder, "2025-07-21 Survey translation*.xlsx")
files = glob.glob(pattern)

def preprocess_html(html_text):
    if pd.isna(html_text):
        return ""
    # Replace <hr /> with <br> for line breaks
    html_text = re.sub(r'<hr\s*/?>', '<br>', str(html_text), flags=re.IGNORECASE)
    return html_text.strip()

for file in files:
    df = pd.read_excel(file)

    doc = Document()
    doc.add_heading(f"Document generated from {os.path.basename(file)}", level=1)

    converter = HtmlToDocx()

    for idx, row in df.iterrows():
        # Add column 0 (index) as a paragraph
        doc.add_paragraph(str(row.iloc[0]), style='Intense Quote')

        # Process columns 1 and 2
        for col_idx in [1, 2]:
            cell_value = row.iloc[col_idx]

            if pd.isna(cell_value) or str(cell_value).strip() == "":
                doc.add_paragraph("")
                continue

            html_content = preprocess_html(cell_value)

            try:
                converter.add_html_to_document(html_content, doc)
            except Exception as e:
                # Fallback: add as plain text if conversion fails
                doc.add_paragraph(str(cell_value))
                print(f"Warning: htmldocx failed on cell with error: {e}")

        # Add a blank paragraph for spacing between rows
        doc.add_paragraph()

    new_doc_name = file.replace('.xlsx', '.docx')
    doc.save(new_doc_name)
    print(f"Created Word document: {new_doc_name}")


Created Word document: translation\2025-07-21 Survey translation CS.docx
Created Word document: translation\2025-07-21 Survey translation DE.docx
Created Word document: translation\2025-07-21 Survey translation ES.docx
Created Word document: translation\2025-07-21 Survey translation FR.docx
Created Word document: translation\2025-07-21 Survey translation IT.docx
Created Word document: translation\2025-07-21 Survey translation NL.docx
Created Word document: translation\2025-07-21 Survey translation PL.docx
Created Word document: translation\2025-07-21 Survey translation PT.docx
Created Word document: translation\2025-07-21 Survey translation RO.docx
